<a href="https://colab.research.google.com/github/aureguerrero/sahi/blob/main/Detectron2_sahi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U git+https://github.com/aureguerrero/sahi.git
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.10/index.html # for Detectron2-cpu
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html # for Detectron2-cuda11.1

exit(0)


In [ ]:
import os
os.getcwd()
# will be used for detectron2 fasterrcnn model zoo name
from sahi.utils.detectron2 import Detectron2TestConstants

# import required functions, classes
from sahi.model import Detectron2DetectionModel
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi.utils.file import download_from_url
from sahi.utils.cv import read_image
from IPython.display import Image
import numpy as np
!pip install rasterio
!pip install affine
!pip install pyproj
!pip install pygeoj
!pip install geopandas
import gdal
import rasterio
from affine import Affine
from pyproj import Proj, transform
import pygeoj
import geopandas as gpd
import matplotlib.pyplot as plt
import cv2
import shutil
import json
import pickle

# set detectron2 fasterrcnn model zoo name
model_path = Detectron2TestConstants.FASTERCNN_MODEL_ZOO_NAME

# download test images into demo_data folder



detection_model = Detectron2DetectionModel(
    model_path=model_path,
    config_path=model_path,
    confidence_threshold=0.7,
    image_size=640,
    device="cpu", # or 'cuda:0'
    category_mapping={'0':'planta'},
)
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/Shareddrives/Conteo de plantas/SEMANTICA/pesos

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
#cfg.merge_from_file(os.path.join(cfg.OUTPUT_DIR, "model_final.pth"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("planta_train",)
cfg.DATASETS.TEST = ()#("planta_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = (os.path.join(cfg.OUTPUT_DIR, "model_final.pth"))
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.DEVICE='cpu'
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
detection_model.model=predictor
%cd /
%cd content/
%mkdir images/
%mkdir resultado/


def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

In [7]:
ancho_ventana=512
alto_ventana=512
solapamiento_ancho=0.2
solapamiento_alto=0.2
prob_de_coinc_pospro=0.6
detection_model.confidence_threshold=0.7 # nivel de confianza para que muestre un objeto como predicción
centroides=1 # si es None no imprime en imagen los centroides si es 1 imprime centroides

In [ ]:
for i in os.listdir('images'):
  fname = '/content/images/'+i
  result= get_sliced_prediction(
    fname,
    detection_model,
    slice_height = alto_ventana,
    slice_width = ancho_ventana,
    overlap_height_ratio = solapamiento_alto,
    overlap_width_ratio = solapamiento_ancho,
    postprocess_match_threshold= prob_de_coinc_pospro,
    )
  result.export_visuals(export_dir='resultado/',export_file='predict_'+i[:-4],centro=centroides)

  with open('resultado/predict_'+i[:-4],'wb') as planta:
    pickle.dump(result,planta,protocol=pickle.HIGHEST_PROTOCOL) 

# Para abrir el archivo con resultados
  #  with open('resultado/predict_'+i[:-4],'rb') as planta:
  #   result=pickle.load(planta)

  # Ver los centroides
  # result.centroides()

  if i[-3:]=='tif' or i[-4:]=='tiff':
    with rasterio.open(fname) as r:
      T0 = r.transform  # upper-left pixel corner affine transform
      p1 = Proj(r.crs)
      A = r.read()
    datos={"type": "FeatureCollection","name": i+'_vectorizacion',"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:"+r.crs.to_string() } }, "features":[]}
    centros={"type": "FeatureCollection","name": i+'_vectorizacion',"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:"+r.crs.to_string() } }, "features":[]}

    id=0
    for obj,c in zip(result.object_prediction_list,result.centroides()):
      img=np.uint8(255*obj.mask.bool_mask)
      contours, hier = cv2.findContours(rotate_bound(cv2.flip(rotate_bound(img, 90),0),180), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cont=np.array(contours).squeeze()
      if len(np.shape(cont))==1:
        planta=[]
        area=0
        for arreglo in cont:
          if len(arreglo)>2:
            arreglo=arreglo.squeeze()
            area=area+cv2.contourArea(arreglo)*abs(T0[0]*T0[4])
            arreglo=np.append(arreglo,[arreglo[0,:]],axis=0)
            arreglo=(arreglo+np.array([obj.bbox.miny,obj.bbox.minx]))*np.array([T0[4],T0[0]])+np.array([T0[5],T0[2]])
            arreglo=[[arreglo[i][1],arreglo[i][0]] for i in range(np.shape(arreglo)[0])]
            planta.append(arreglo)
        datos['features'].append({ "type": "Feature", "properties": { "CLASS_NAME": "planta", "AREA": area }, "geometry": { "type": "MultiPolygon", "coordinates":[planta]}})

      else:
        area=cv2.contourArea(cont)*abs(T0[0]*T0[4])
        cont=np.append(cont,[cont[0,:]],axis=0)
        cont=(cont+np.array([obj.bbox.miny,obj.bbox.minx]))*np.array([T0[4],T0[0]])+np.array([T0[5],T0[2]])
        cont=[[cont[i][1],cont[i][0]] for i in range(np.shape(cont)[0])]
        datos['features'].append({ "type": "Feature", "properties": { "CLASS_NAME": "planta", "AREA": area }, "geometry": { "type": "MultiPolygon", "coordinates":[[cont]]}})
      centros['features'].append({ "type": "Feature", "properties": { "CLASS_NAME": "planta", "AREA": area, "ID" :id}, "geometry": { "type": "Point", "coordinates":[c[0]*T0[4]+T0[5],c[1]*T0[0]+T0[2]]}})
      id=id+1
    with open('/content/resultado/'+i[:-4]+'_vectorial.geojson', 'w') as fp:
      json.dump(datos, fp)
    with open('/content/resultado/'+i[:-4]+'_centroides.geojson', 'w') as fp:
      json.dump(centros, fp)



In [ ]:
!zip -r resultados.zip resultado/